In [1]:
import pandas as pd
import numpy as np
from pandas_datareader import data as pdr
import pandas as pd
import yfinance as yf
import yahoofinancials
import seaborn as sns
import datetime
from datetime import date
import matplotlib.pyplot as plt

import random
from random import randrange
from scipy.stats import norm

import scipy.stats as si
import sympy as sy
from sympy.stats import Normal, cdf
from sympy import init_printing
init_printing()

In [2]:
# European Option Price 

In [3]:
def euro_vanilla(S, K, T, r, sigma, option = 'call'):
    
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = (np.log(S / K) + (r - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    
    if option == 'call':
        result = (S * si.norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * T) * si.norm.cdf(d2, 0.0, 1.0))
    if option == 'put':
        result = (K * np.exp(-r * T) * si.norm.cdf(-d2, 0.0, 1.0) - S * si.norm.cdf(-d1, 0.0, 1.0))
        
    return result

In [4]:
# User buys option and it is added to the batch of all options
# weighted average of Options 
def pushOption(amount,payment, S,K,T,R,SIGMA, DELTA):
    A = options["amount"]
    options["amount"] += amount
    
    options["payment"] += payment
    
    S = (amount * S + A * options["S"]) / (amount + A)
    options["S"] = S
    
    K = (amount * K + A * options["K"]) / (amount + A)
    options["K"] = K
    
    T = (amount * T + A * options["T"]) / (amount + A)
    options["T"] = T
    
    R = (amount * R + A * options["R"]) / (amount + A)
    options["R"] = R
    
    SIGMA = (amount * SIGMA + A * options["SIGMA"]) / (amount + A)
    options["SIGMA"] = SIGMA


In [5]:
def BuyOption(price):

    amount = random.randint(1,5)
    
    S = price
    K = price - random.randint(0,200)
    T = 1
    r = 0.1
    sigma = 0.5
    
    option = "call"
    
    P = euro_vanilla(S,K,T,r,sigma, option)
    
    price = P * amount

    pushOption(amount,price,S,K,T,r,sigma, option)
    

In [6]:
def delta_calc(S, K, T, r, sigma):
    
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    delta_calc = norm.cdf(d1, 0, 1)

    return delta_calc

In [7]:
def UpdateDelta():
    
    S = options["S"]
    K = options["K"]
    T = options["T"]
    r = options["R"]
    sigma = options["SIGMA"]
    
    DELTA = delta_calc(S,K,T,r,sigma)
    
    options["DELTA"] = DELTA
    
    return DELTA

In [8]:
# initializing batch of all options 
options = {"amount": 0, "payment": 0, "S": 0, "K": 0, "T": 0, "R": 0, "SIGMA": 0, "DELTA": 0}

In [9]:
# download historical data
yf.pdr_override() 
df = pdr.get_data_yahoo("ETH-USD", start="2022-01-01", end="2022-03-12")
df.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-12-31,3713.430176,3807.288818,3636.869873,3682.632812,3682.632812,14157285268
2022-01-01,3683.047119,3769.917969,3682.286621,3769.697021,3769.697021,9776191466
2022-01-02,3769.298340,3836.161865,3727.357422,3829.564941,3829.564941,9881471548
2022-01-03,3829.535645,3836.198730,3698.047607,3761.380371,3761.380371,12080777893
2022-01-04,3761.361572,3876.785156,3723.349854,3794.056641,3794.056641,14030925983


In [10]:
df = df.reset_index()

In [11]:
for index, row in df.iterrows():
    price = row["Close"]
    
    BuyOption(price)
    UpdateDelta()

In [12]:
options

{'amount': 222,
 'payment': 166318.66634319324,
 'S': 2951.0867436039557,
 'K': 2855.915572432785,
 'T': 1.0,
 'R': 0.10000000000000006,
 'SIGMA': 0.5,
 'DELTA': 0.6969198271068432}

In [ ]:
## Logic of DeltaHedge.sol

In [13]:
def rebalancePool(USDC, ETH, rate, delta): 
    ethToUSDC = ETH * rate
    poolVal = ethToUSDC + USDC
    
    deltaNow = ethToUSDC / poolVal
    
    print("Pool Value " + str(poolVal))
    print("Delta Now " + str(deltaNow))
    
    if (deltaNow < delta):
        
        newAmount = delta * poolVal / rate
        amount = newAmount - ETH
        
        swap = amount * rate
        
        print("Swap " + str(swap) + " USDC for " + str(amount) + " ETH")
    
        print("Buy " + str(amount) + " ETH")
        
        
    if (deltaNow > delta):
        
        newAmount = delta * poolVal / rate
        amount = ETH - newAmount
    
        swap = amount * rate
        
        print("Swap " + str(amount) + " ETH for " + str(swap) + " USDC")

        print("Sell " + str(amount) + " ETH")

In [16]:
rebalancePool(300000,100,3000,0.9)

Pool Value 600000
Delta Now 0.5
Swap 240000.0 USDC for 80.0 ETH
Buy 80.0 ETH


In [17]:
rebalancePool(480000,40,3000,0.2)

Pool Value 600000
Delta Now 0.2


In [ ]:
# plot a random delta on df 
# not connected to code above... 

In [ ]:
df["Delta"] = delta_calc(df["Close"],(df["Close"] - random.randint(-100,300)),1,0.1,0.5)

In [ ]:
df.head()

In [ ]:
import matplotlib.pyplot as plt

df.plot(kind='line',x='Date',y='Delta',color='red')
plt.show()

In [ ]:
df.plot(kind='line',x='Date',y='Close',color='red')
plt.show()

In [ ]:
############ LIQUIDITY POOL #################

In [ ]:
S = options["S"]
K = options["K"]
T = options["T"]
r = options["R"]
sigma = options["SIGMA"]

In [ ]:
BS = euro_vanilla(S,K,T,r,sigma)
print(BS)

In [ ]:
delta_calc(S,K,T,r,sigma)

In [ ]:
################ END OF POLISHED CODE #################
############ From here on is just random test junk ###########

In [ ]:
S = 100
K = 100
T = 1
r = 0.07
sigma = 0.5
option = "call"

In [ ]:
euro_vanilla(S,K,T,r,sigma, option)

In [ ]:
S = 100
K = 100
T = 1
r = 0.5
sigma = 0.7
option = "call"

In [ ]:
euro_vanilla(S,K,T,r,sigma, option)

In [ ]:
def CallPrice(price):
    S = price
    K = S - 100
    T = 1
    r = 0.1
    sigma = 0.3
    option = "call"
    return euro_vanilla(S,K,T,r,sigma, option)


def PutPrice(price):
    S = price
    K = S + 100
    T = 1
    r = 0.1
    sigma = 0.3
    option = "put"
    return euro_vanilla(S,K,T,r,sigma, option)
    

def weightedP(num_games, prob_heads=.5): 
    player_one_wins = 0 
    for n in range(0,num_games): 
        num_flips = 0 
        win = 0
        while win == 0: 
            turn = np.random.uniform(0,1) 
            num_flips += 1 
            if turn <= prob_heads: 
                if num_flips % 2 != 0: 
                    player_one_wins += 1 
                win += 1 
    return float(player_one_wins)/float(num_games)
    
    
    
def UserDecision(price):
    
    pool1 = df['Pool1'].iloc[-1]
    pool2 = df['Pool2'].iloc[-1]
    
    total = pool1 + pool2
    val = pool1/total
    decision = weightedP(1, val)
    
    return decision
    


In [ ]:
df['Pool1'] = 10
df['Pool2'] = 10

In [ ]:
df.head()

In [ ]:
#for i in range(len(df['Close'])):

for i in range(5):
    
    price = df["Close"].iloc[i]
    
    pool1 = df["Pool1"].iloc[i]
    pool2 = df["Pool2"].iloc[i]

    total = pool1 + pool2
    val = pool1/total
    

    
    decision = weightedP(1, val)
    
    print("Pool1: ", pool1)
    print("Pool1: ", pool2)
    print("val: ", val)
    print("decision: ", decision)
    
    if decision == 0:
        # call
        print("call")
        
        pool1 += CallPrice(price)
        
        df["Pool1"].iloc[i] = pool1
        df["Pool1"].iloc[i] = pool2
    
    else: 
        # put
        
        print("put")
        pool2 += PutPrice(price)
        
        df["Pool1"].iloc[i] = pool1
        df["Pool1"].iloc[i] = pool2
        
    

In [ ]:
pool1 = df["Pool1"].iloc[0]
print(pool1)

In [ ]:
pool1 = df["Pool1"].iloc[0]
pool2 = df["Pool2"].iloc[0]

total = pool1 + pool2
val = pool1/total
decision = weightedP(1, val)
print(val)
print(decision)

In [ ]:
pool1 = df['Pool1'].iloc[-1]
print(pool1)

In [ ]:
df['Pool1'] = 100
df['Pool2'] = 100

In [ ]:
for i in df.index:
        df.at[i, 'ifor'] = x

In [ ]:
df

In [ ]:
def UpdatePool(pool1, pool2, price):
    
    UserDecision(pool1, pool2, price)
    
    if decision == 0:
        option = "call"
        pool1 += CallPrice(price)
        return pool1
        
    else: 
        option = "put"
        pool2 += PutPrice(price)
        return pool2

    
    
CallPool = pd.Series(CallPool(df["Close"]), name='CallPrice')


In [ ]:
UserDecision(50,50,3000)

In [ ]:
weightedP(1, 0.9149591015180875)

In [ ]:
CallPrice = pd.Series(CallPrice(df["Close"]), name='CallPrice')
PutPrice = pd.Series(PutPrice(df["Close"]), name='PutPrice')
df = df.join(CallPrice)
df = df.join(PutPrice)

In [ ]:
df

In [ ]:
CallPool = pd.Series()

In [ ]:
df

In [ ]:
d = ((df['close_rate'] - df['open_rate']) / df['open_rate']) * 100
delta = pd.Series(d, name = 'delta') 
    
    
pool1 = OptionPrice(df["Close"]) 
    

In [ ]:
df["Close"]

In [ ]:
CallPoolVal = PutPool/CallPool
PutPoolVal = CallPool/PutPool

In [ ]:

def weightedP(num_games, prob_heads=.5): 
    player_one_wins = 0 
    for n in range(0,num_games): 
        num_flips = 0 
        win = 0
        while win == 0: 
            turn = np.random.uniform(0,1) 
            num_flips += 1 
            if turn <= prob_heads: 
                if num_flips % 2 != 0: 
                    player_one_wins += 1 
                win += 1 
    return float(player_one_wins)/float(num_games)

In [ ]:
weightedP(10, 0.5)

In [ ]:
pool1 = 551.2345061060226
pool2 = 51.2345061060226

total = pool1 + pool2

val = pool1/total
val